In [1]:
import os
os.chdir("..")

In [2]:
import asyncio
from dotenv import load_dotenv
import shutil
import subprocess
import requests
import time
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
 

class LanguageModelProcessor:
    def __init__(self,prompt):
        self.llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768", groq_api_key=os.getenv("GROQ_API_KEY"))
 

        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

        

        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{text}")
        ])

        self.conversation = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )

    def process(self, text):
        self.memory.chat_memory.add_user_message(text)  # Add user message to memory

        start_time = time.time()

        # Go get the response from the LLM
        response = self.conversation.invoke({"text": text})
        end_time = time.time()

        self.memory.chat_memory.add_ai_message(response['text'])  # Add AI response to memory

        elapsed_time = int((end_time - start_time) * 1000)
        # print(f"LLM ({elapsed_time}ms): {response['text']}")
        return response['text']  


In [4]:
prompt = """Given the following information:

1. Folder Name: {folder_name}
2. JSON File Name: user_info.json
3. Current Chat History: 
   {chat_history}

Task:
* Check if user_info.json exists in the specified folder.
* IF JSON EXISTS:
    * Extract name and relevant past history.
    * Craft a personalized response (max 2 lines) based on the current chat and history.
    * Prioritize using the name and a warm greeting.
* IF JSON DOES NOT EXIST:
    * Craft a general, friendly response (max 2 lines) to continue the conversation.

Example JSON (if exists):
{
  "name": "Alice",
  "past_history": ["Asked about product features", "Expressed interest in discounts"]
}"""

In [3]:
llm = LanguageModelProcessor(prompt)
llm.process(text)

"I'm just a computer program, but I'm here to help you. How can I assist you today?"